# Evaluating RAG Systems

Evaluation and benchmarking are crucial in developing LLM applications. Optimizing performance for applications like RAG (Retrieval Augmented Generation) requires a robust measurement mechanism.

LlamaIndex provides essential modules to assess the quality of generated outputs and evaluate content retrieval quality. It categorizes its evaluation into two main types:

*   **Response Evaluation** : Assesses quality of Generated Outputs
*   **Retrieval Evaluation** : Assesses Retrieval quality

[Documentation
](https://docs.llamaindex.ai/en/latest/module_guides/evaluating/)

In [1]:
# !pip install llama-index

## Response Evaluation

Evaluating results from LLMs is distinct from traditional machine learning's straightforward outcomes. LlamaIndex employs evaluation modules, using a benchmark LLM like GPT-4, to gauge answer accuracy. Notably, these modules often blend query, context, and response, minimizing the need for ground-truth labels.

The evaluation modules manifest in the following categories:

*   **Faithfulness:** Assesses whether the response remains true to the retrieved contexts, ensuring there's no distortion or "hallucination."
*   **Relevancy:** Evaluates the relevance of both the retrieved context and the generated answer to the initial query.
*   **Correctness:** Determines if the generated answer aligns with the reference answer based on the query (this does require labels).

Furthermore, LlamaIndex has the capability to autonomously generate questions from your data, paving the way for an evaluation pipeline to assess the RAG application.

In [ ]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

from jet.llm.ollama.base import initialize_ollama_settings, Ollama
initialize_ollama_settings()

# Set up the root logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Set logger level to INFO

# Clear out any existing handlers
logger.handlers = []

# Set up the StreamHandler to output to sys.stdout (Colab's output)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)  # Set handler level to INFO

# Add the handler to the logger
logger.addHandler(handler)

In [3]:
import logging
import sys
import pandas as pd

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core.evaluation import (
    DatasetGenerator,
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    CorrectnessEvaluator,
    RetrieverEvaluator,
    generate_question_context_pairs,
)

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    Response,
)

# from llama_index.llms.openai import OpenAI

import os

In [4]:
# os.environ["OPENAI_API_KEY"] = "sk-..."

#### Download Data

In [5]:
# !mkdir -p 'data/paul_graham/'
# !wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

#### Load Data

In [6]:
reader = SimpleDirectoryReader("/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/JetScripts/data/jet-resume/data/")
documents = reader.load_data()

#### Generate Question

In [ ]:
gpt4 = Ollama(model="llama3.1", temperature=0.1)

dataset_generator = DatasetGenerator.from_documents(
    documents,
    llm=gpt4,
    show_progress=True,
    num_questions_per_chunk=2,
)

eval_dataset = dataset_generator.generate_dataset_from_nodes(num=1)

In [8]:
eval_queries = list(eval_dataset.queries.values())

In [ ]:
(eval_queries)

In [ ]:
len(eval_queries)

To be consistent we will fix evaluation query

In [11]:
eval_query = "How did the author describe their early attempts at writing code?"

In [12]:
# Fix GPT-3.5-TURBO LLM for generating response
gpt35 = Ollama(temperature=0, model="llama3.2")

# Fix GPT-4 LLM for evaluation
gpt4 = Ollama(temperature=0, model="llama3.1")

In [ ]:
# create vector index
vector_index = VectorStoreIndex.from_documents(documents, llm=gpt35)

# Query engine to generate response
query_engine = vector_index.as_query_engine()

In [ ]:
retriever = vector_index.as_retriever(similarity_top_k=3)
nodes = retriever.retrieve(eval_query)

In [ ]:
from IPython.display import display, HTML

display(HTML(f'<p style="font-size:20px">{nodes[1].get_text()}</p>'))

#### Faithfullness Evaluator

 Measures if the response from a query engine matches any source nodes. This is useful for measuring if the response was hallucinated.

In [16]:
faithfulness_evaluator = FaithfulnessEvaluator(llm=gpt4)

In [ ]:
# Generate response
response_vector = query_engine.query(eval_query)

In [ ]:
eval_result = faithfulness_evaluator.evaluate_response(
    response=response_vector
)

In [ ]:
eval_result.passing

In [ ]:
eval_result

#### Relevency Evaluation

Measures if the response + source nodes match the query.

In [21]:
# Create RelevancyEvaluator using GPT-4 LLM
relevancy_evaluator = RelevancyEvaluator(llm=gpt4)

In [ ]:
# Generate response
response_vector = query_engine.query(eval_query)

# Evaluation
eval_result = relevancy_evaluator.evaluate_response(
    query=eval_query, response=response_vector
)

In [ ]:
eval_result.query

In [ ]:
eval_result.response

In [ ]:
eval_result.passing

Relevancy evaluation with multiple source nodes.

In [ ]:
# Create Query Engine with similarity_top_k=3
query_engine = vector_index.as_query_engine(similarity_top_k=3)

# Create response
response_vector = query_engine.query(eval_query)

# Evaluate with each source node
eval_source_result_full = [
    relevancy_evaluator.evaluate(
        query=eval_query,
        response=response_vector.response,
        contexts=[source_node.get_content()],
    )
    for source_node in response_vector.source_nodes
]

# Evaluation result
eval_source_result = [
    "Pass" if result.passing else "Fail" for result in eval_source_result_full
]

In [ ]:
eval_source_result

#### Correctness Evaluator

Evaluates the relevance and correctness of a generated answer against a reference answer.

In [28]:
correctness_evaluator = CorrectnessEvaluator(llm=gpt4)

In [29]:
query = "Can you explain the theory of relativity proposed by Albert Einstein in detail?"

reference = """
Certainly! Albert Einstein's theory of relativity consists of two main components: special relativity and general relativity. Special relativity, published in 1905, introduced the concept that the laws of physics are the same for all non-accelerating observers and that the speed of light in a vacuum is a constant, regardless of the motion of the source or observer. It also gave rise to the famous equation E=mc², which relates energy (E) and mass (m).

General relativity, published in 1915, extended these ideas to include the effects of gravity. According to general relativity, gravity is not a force between masses, as described by Newton's theory of gravity, but rather the result of the warping of space and time by mass and energy. Massive objects, such as planets and stars, cause a curvature in spacetime, and smaller objects follow curved paths in response to this curvature. This concept is often illustrated using the analogy of a heavy ball placed on a rubber sheet, causing it to create a depression that other objects (representing smaller masses) naturally move towards.

In essence, general relativity provided a new understanding of gravity, explaining phenomena like the bending of light by gravity (gravitational lensing) and the precession of the orbit of Mercury. It has been confirmed through numerous experiments and observations and has become a fundamental theory in modern physics.
"""

response = """
Certainly! Albert Einstein's theory of relativity consists of two main components: special relativity and general relativity. Special relativity, published in 1905, introduced the concept that the laws of physics are the same for all non-accelerating observers and that the speed of light in a vacuum is a constant, regardless of the motion of the source or observer. It also gave rise to the famous equation E=mc², which relates energy (E) and mass (m).

However, general relativity, published in 1915, extended these ideas to include the effects of magnetism. According to general relativity, gravity is not a force between masses but rather the result of the warping of space and time by magnetic fields generated by massive objects. Massive objects, such as planets and stars, create magnetic fields that cause a curvature in spacetime, and smaller objects follow curved paths in response to this magnetic curvature. This concept is often illustrated using the analogy of a heavy ball placed on a rubber sheet with magnets underneath, causing it to create a depression that other objects (representing smaller masses) naturally move towards due to magnetic attraction.
"""

In [ ]:
correctness_result = correctness_evaluator.evaluate(
    query=query,
    response=response,
    reference=reference,
)

In [ ]:
correctness_result

In [ ]:
correctness_result.score

In [ ]:
correctness_result.passing

In [ ]:
correctness_result.feedback

## Retrieval Evaluation

Evaluates the quality of any Retriever module defined in LlamaIndex.

To assess the quality of a Retriever module in LlamaIndex, we use metrics like hit-rate and MRR. These compare retrieved results to ground-truth context for any question. For simpler evaluation dataset creation, we utilize synthetic data generation.

In [36]:
reader = SimpleDirectoryReader("/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/JetScripts/data/jet-resume/data")
documents = reader.load_data()

from llama_index.core.text_splitter import SentenceSplitter

# create parser and parse document into nodes
parser = SentenceSplitter(chunk_size=1024, chunk_overlap=100)
nodes = parser(documents)

In [ ]:
vector_index = VectorStoreIndex(nodes)

In [38]:
# Define the retriever
retriever = vector_index.as_retriever(similarity_top_k=2)

In [ ]:
retrieved_nodes = retriever.retrieve(eval_query)

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

for node in retrieved_nodes:
    display_source_node(node, source_length=2000)

In [ ]:
qa_dataset = generate_question_context_pairs(
    nodes, llm=gpt4, num_questions_per_chunk=2
)

In [ ]:
queries = qa_dataset.queries.values()
print(list(queries)[5])

In [ ]:
len(list(queries))

In [44]:
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever
)

In [ ]:
# try it out on a sample query
sample_id, sample_query = list(qa_dataset.queries.items())[0]
sample_expected = qa_dataset.relevant_docs[sample_id]

eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
print(eval_result)

In [ ]:
# try it out on an entire dataset
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

In [47]:
def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"retrievers": [name], "hit_rate": [hit_rate], "mrr": [mrr]}
    )

    return metric_df

In [ ]:
display_results("top-2 eval", eval_results)